# Plugin Profiler Tutorial

This notebook demonstrates how to use the appletree profiler to analyze the performance of individual plugins in the simulation chain. This is useful for:

- Identifying performance bottlenecks
- Understanding the execution flow of plugins
- Comparing individual plugin timing vs the full JIT-fused pipeline

In [ ]:
import appletree as apt
from appletree.utils import get_file_path

## 1. Initialize Context

First, we create a context with the Rn220 configuration.

In [ ]:
config = get_file_path("rn220.json")
tree = apt.Context(config)

## 2. View Plugin Execution Order

Before profiling, let's understand the plugin execution order (worksheet) for a component.

In [ ]:
component = tree.likelihoods["rn220_llh"].components["rn220_er"]
apt.profiler.print_worksheet(component)

## 3. View Generated Code

We can also inspect the auto-generated JAX code that appletree creates from the plugin chain.

In [ ]:
apt.profiler.print_component_code(component)

## 4. Profile All Plugins

Now let's profile each plugin individually. The profiler will:
1. Warm up each plugin (JIT compilation)
2. Run multiple timed iterations
3. Report mean and standard deviation of execution time

**Parameters:**
- `batch_size`: Number of events to simulate
- `n_warmup`: Number of warmup runs for JIT compilation
- `n_runs`: Number of timed runs

In [ ]:
results = apt.profile_context(
    tree,
    batch_size=1_000_000,
    n_warmup=2,
    n_runs=10,
)

## 5. Compare Individual Plugins vs Full Pipeline

JAX's JIT compiler can fuse operations together when compiling the full simulation chain, which can be faster than running each plugin separately. Let's compare:

In [ ]:
apt.compare_plugin_vs_full(
    tree,
    batch_size=1_000_000,
    n_warmup=2,
    n_runs=10,
)

The positive overhead indicates that running plugins individually is slower than the full JIT-fused pipeline. This is expected because JAX can optimize memory access patterns and fuse operations when compiling everything together.

## 6. Profile a Single Component

You can also profile a single component directly if you want more control.

In [ ]:
# Get parameters
tree.par_manager.sample_init()
parameters = tree.par_manager.get_all_parameter()

# Profile single component
component = tree.likelihoods["rn220_llh"].components["rn220_er"]
results = apt.profile_component(
    component,
    parameters,
    batch_size=1_000_000,
    n_warmup=2,
    n_runs=5,
)

## 7. Analyze Results Programmatically

The profiler returns structured results that you can analyze further.

In [ ]:
# Sort plugins by execution time
sorted_results = sorted(results, key=lambda x: x["mean_time_ms"], reverse=True)

print("Plugins sorted by execution time:")
print("-" * 50)
for r in sorted_results[:10]:
    print(f"{r['plugin']:<30} {r['mean_time_ms']:.3f} ms")

In [ ]:
# Calculate total time and percentage breakdown
total_time = sum(r["mean_time_ms"] for r in results)

print(f"\nTotal time: {total_time:.3f} ms")
print("\nTop 5 plugins by percentage:")
print("-" * 50)
for r in sorted_results[:5]:
    pct = r["mean_time_ms"] / total_time * 100
    print(f"{r['plugin']:<30} {pct:.1f}%")